OPENAI API KEY 와 TAVILY API KEY 필요

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

#### **기억력 만들기**
이전에 만든 인터넷 검색 Tool 코드에 이어 기억력 추가

In [2]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph

# MemorySaver는 in memory 상에서 그래프의 기억력이 유지되게끔 해주는 모듈
# 만약 직접 저장소를 구축해서 사용자의 메세지를 지속해서 기록하고 저장하고 싶다면 SqlLiteMemorySaver 이런식으로 langgraph에서 지원하는 라이브러리를 사용하면된다.
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver() # MemorySaver()를 memory에 할당 ()

# 아래 코드는 이전 코드랑 같음
class State(TypedDict):
    messages: Annotated[list, add_messages]

tool = TavilySearchResults(max_results=2)
tools = [tool]
tool_node = ToolNode(tools)

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    result = llm_with_tools.invoke(state["messages"])
    return {"messages": [result]}

graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

graph_builder.add_edge("tools", "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition)

graph_builder.set_entry_point("chatbot")
graph = graph_builder.compile(checkpointer=memory) # 마지막 컴파일을 할 때 메모리를 checkpointer로 설정을 해주면 된다. 특정 쓰레드에 한해서 대화의 기록이 유지된다.

In [3]:
config = {"configurable": {"thread_id": "1"}} # 대화의 쓰레드를 id로 지정하는것. (thread_id 1번 에다 대화내용을 저장할건데 이 부분에 관련해서 그래프에서 나눈 대화가 계속해서 유지가 되게끔 만들어줌)

# 이전의 대화내용을 기억하는지 실행해서 확인해보자
while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break
    for event in graph.stream({"messages": ("user", user_input)}, config):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

Assistant: 
Assistant: [{"url": "https://namu.wiki/w/대한민국+대통령", "content": "대한민국 대통령은 대한민국의 대통령 기관(機關)·직위(職位) 및 이를 수행하는 인물을 가리킨다. 대한민국의 현직 대통령은 제20대 윤석열이며, 임기는 2022년 5월 10일부터 2027년 5월 9일까지이다. 대통령은 대한민국 헌법 제66조에서 정의하는 헌법기관으로, 국가원수 [원]로서 외국에 대하여 국가를"}, {"url": "https://ko.wikipedia.org/wiki/대한민국의_대통령_목록", "content": "개인 도구 대한민국의 대통령 목록 도구 위키백과, 우리 모두의 백과사전. 제9차 개헌 이후의 현행 헌법에 따라 대통령의 임기는 5년이며 중임할 수 없지만,[1] 제9차 개헌 이전의 여러 헌법에서는 대통령의 임기가 고정되지 않았으며, 그로 인해 대통령직을 수행한 개인과 대통령 임기의 대수가 불일치하게 되었다. (생몰년)   정당  대[b]    임기  선거 / 득표율[5] ↑ 가 나 2016년 헌법에 위배되는 박근혜-최순실 게이트, 비선실세 의혹, 대기업 뇌물 의혹 등을 사유로 박근혜에 대한 탄핵소추안이 국회에서 가결되었고, 대통령 권한이 정지되어 황교안 국무총리가 권한대행을 맡았다.[27] 결과적으로 2017년 3월 10일 박근혜는 헌법재판소에 의해 전원일치로 대통령직에서 파면되었다.[28][29] ↑ 허정 (1979). ↑ KBS 뉴스 (2004). ↑ KBS 뉴스 (2017). ↑ 중앙일보 후보자 정보 (윤석열). 14 김영삼15 김대중16 노무현17 이명박18 박근혜19 문재인20 윤석열 대한민국의 대통령과 3부요인 및 헌법기관장 분류: 알찬 목록 후보 문서대한민국의 대통령대한민국 정치에 관한 목록대통령 목록대한민국 사람 목록 위키백과 소개"}]
Assistant: 현재 대한민국의 대통령은 윤석열입니다. 그는 2022년 5월 10일부터 2027년 5월 9일까지 임기를 수행하고 있습니다. 더 자세한 

In [4]:
# 실제로 대화 내용들이 어떤식으로 저장이 되고 있는지 알아보자
snapshot = graph.get_state(config) # get_state함수에 config를 넣어줘서 확인
snapshot

# 아래 결과를 보면 configurable의 thread_id 1번에 저장 돼있던 이 대화 메세지들을 전부 가져옴
# 첫번째, 두번째, 세번째 질문이 전부 나온걸 볼 수 있음.
# 이렇게 state message에 저장된 내용들을 계속해서 메모리로 활용을 하면서 대화의 맥락을 유지할 수 있다.

StateSnapshot(values={'messages': [HumanMessage(content='대한민국의 대통령은 누구야?', additional_kwargs={}, response_metadata={}, id='53f340f8-df72-442c-bf7a-76ff079a1441'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_6HWy9kjpm0DP48ciw5S6EXM1', 'function': {'arguments': '{"query":"현재 대한민국 대통령"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 87, 'total_tokens': 108, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_39a40c96a0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f1c793fa-2b88-4a69-aca6-9991e77ce31b-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '현재 대한민국 대통령'}, 'id': 'call_6HWy9kjpm0DP48ciw5S6EXM1',

### 그럼 thread_id를 다르게 배정하면 메모리가 기억을 하는지 확인해보자

In [ ]:
config2 = {"configurable": {"thread_id": "2"}}
graph.invoke({"messages": [{"role": "user", "content": "내가 한 첫 질문이 뭐였어?"}]},
             config2)

# 이전 대화를 기억할 수 없다면서 솔직하게 말하는걸 볼 수 있다. thread_id: 2에서는 이전 대화를 기억하지 못한다.

{'messages': [HumanMessage(content='내가 한 첫 질문이 뭐였어?', additional_kwargs={}, response_metadata={}, id='6351dcb0-ca60-448f-91c2-915e4e743bda'),
  AIMessage(content='죄송하지만, 이전 대화 내용이나 질문을 기억할 수 없습니다. 하지만 새로운 질문이나 도움이 필요한 내용을 말씀해 주시면 최선을 다해 도와드리겠습니다!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 89, 'total_tokens': 128, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_39a40c96a0', 'finish_reason': 'stop', 'logprobs': None}, id='run-29d1d23f-724d-4ac2-8203-8f427aaf3ec8-0', usage_metadata={'input_tokens': 89, 'output_tokens': 39, 'total_tokens': 128, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}

State Message에 계속해서 내용들을 쌓아나가는 순간 LLM에게 State Message를 전부 보내게 되면

대화 내용이 엄청 긴 경우에 LLM이 전부 받아들이지 못해 에러가 나는 경우가 생길 수 있다.

Context Window가 정해져 있기 때문에 대화가 무한정으로 길어지게 되면 모든 대화 맥락을 기억할 수가 없다.

에러가 날 수 있기 때문에 에이전트 시스템에 불안정한 요소를 가져온다.

#### **기억할 메세지 개수 제한하기**

그래서 최근 몇개 까지만 기억할 수 있도록 설정해보자.

In [6]:
def filter_messages(messages: list):
    # This is very simple helper function which only ever uses the last message
    return messages[-2:] # 가장 마지막 2개 메세지만 가져오도록 함, (마지막 2개 대화까지만 기억하는 헬퍼함수)
    
def chatbot(state: State):
    messages = filter_messages(state["messages"]) # filter_messages 함수를 적용해서 모든 state의 "messages"를 가져오는게 아닌 최근 2개까지만 가져오도록 함
    result = llm_with_tools.invoke(messages) # 최근 2개의 대화만 가져와서 답변을 해줌
    return {"messages": [result]}

graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

graph_builder.add_edge("tools", "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition)

graph_builder.set_entry_point("chatbot")
graph = graph_builder.compile(checkpointer=memory)

In [10]:
from langchain_core.messages import HumanMessage

# 첫번째 질문이 들어갔을 때의 대답과
config = {"configurable": {"thread_id": "20"}}
input_message = HumanMessage(content="hi! I'm bob and I like soccer")
for event in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

# This will now not remember the previous messages
# (because we set `messages[-1:]` in the filter messages argument)

# 두번째 질문이 들어갔을 때의 대답
input_message = HumanMessage(content="what's my name?")
for event in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

# 세번째 질문이 들어갔을 때의 대답
input_message = HumanMessage(content="what's my name?")
for event in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

# 네번째 질문이 들어갔을 때의 대답
input_message = HumanMessage(content="what's my favorite?")
for event in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

# 네번째의 질문은 첫번째 질문을 기억해야지만 대답을 할 수 있는 질문이다.
# 최근 2개의 대화까지만 기억을 하도록 만들었기 떄문에 첫번째 대화는 기억을 못할 것이다.
# 한번 실행해보자.

================================ Human Message =================================

hi! I'm bob and I like soccer
================================== Ai Message ==================================

Nice to meet you, Bob! Soccer is a fantastic sport. Do you have a favorite team or player?
================================ Human Message =================================

what's my name?
================================== Ai Message ==================================

Your name is Bob.
================================ Human Message =================================

what's my name?
================================== Ai Message ==================================

Your name is Bob.
================================ Human Message =================================

what's my favorite?
================================== Ai Message ==================================

I don’t have information about your personal preferences or favorites. If you let me know what specific category you’re referring to (l

마지막 질문은 대답하지 못하는걸 알 수 있다.

이런식으로 커스텀 함수로 State Message에서 특정한 값만 가져와서 저장을 하도록 할 수도 있다.

filter_messages() 함수로 특정 값만 기억하도록 하는 대신에

메세지들을 전부 받아와서 summary에 저장해두고, 즉 요약을 해서 특정 키값에 저장하고
이걸 LLM에게 보내서 대화를 유지하게끔 만들 수 있다.

내가 원하는 형태의 메세지 관리를 다양한 방법으로 할 수 있고, 좀 더 고도화된 랭그래프를 만들 수 있다.